In [ ]:
'''this is a script designed to take stopping tables exported from SRIM and reformat
and convert them into a csv file that LAMMPS can read to add electronic stopping power to an MD simulation'''

#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime
from array import array
import sys
from time import sleep
import scipy
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy import interpolate
import re

In [ ]:
# @author: wuaudrey
def reformat(file):
  with open(file, 'r') as f:
    lines = f.readlines()

    # strip whitespace and split each line into a list of numbers (as strings)
    data = [line.strip().split() for line in lines]

    # === Trim lines from top and bottom ===
    ''' num_atom_types should equal the number of atoms in your crystal.
    for example, SiO2 has two types of atoms, so num_atom_types = 2.0.
    NOTE: if your SRIM file was made using Si(1) + O(1) + O(1) for example, your num_atom_types should be 3.0.
    however your atoms are formatted in the SRIM application should be the format you use to determine
    how many 'atom types' you have in your crystal'''

    num_atom_types = 2
    top_trim = 22 + num_atom_types
    bottom_trim = 13
    data = data[top_trim:len(data)-bottom_trim]

    # remove unneeded columns and convert first column to eV
    for i, line in enumerate(data):
          del line[4], line[5]
          del line[4], line[5]
          del line[4]

          if len(line) > 4:
            del line[4]

          if line[1] == 'eV':
            del line[1]

          if line[1] == 'keV':
            line[0] = str(float(line[0]) * 1000)
            del line[1]

          if line[1] == 'MeV':
            line[0] = str(float(line[0]) * 1000000)
            del line[1]

  output_file = file.replace(".txt", "_fixed.txt")

  with open(output_file, "w") as f:
    for line in data:
        f.write(" ".join(line) + "\n")

  return output_file

reformat('Au_in_SiO2_500keV.txt')
reformat('O_in_SiO2_500keV.txt')
reformat('Si_in_SiO2_500keV.txt')

In [ ]:
# @author: emlavoie

filename_au = 'Au_in_SiO2_500keV_fixed.txt'
#filename_mg = 'Magnesium in Olivine.txt'
#filename_fe = 'Iron in Olivine.txt'
filename_o = 'O_in_SiO2_500keV_fixed.txt' #'Si_in_SiO2/O_1MeV.txt'
filename_si = 'Si_in_SiO2_500keV_fixed.txt' #'Si_in_SiO2/Si_1MeV.txt'

# =============================================================================
# # stopping table from SRIM
# fin_au = "stopping power tables/" + filename_au
# fin_mg = "stopping power tables/" + filename_mg
# fin_fe = "stopping power tables/" + filename_fe
# fin_o = "stopping power tables/" + filename_o
# fin_si = "stopping power tables/" + filename_si
# =============================================================================

df_au = pd.read_csv(filename_au, index_col=None, header=None, sep='\s+')
# =============================================================================
#df_mg = pd.read_csv(fin_mg, index_col=None, header=None, sep='\s+')
#df_fe = pd.read_csv(fin_fe, index_col=None, header=None, sep='\s+')
# =============================================================================
df_o = pd.read_csv(filename_o, index_col=None, header=None, sep='\s+')
df_si = pd.read_csv(filename_si, index_col=None, header=None, sep='\s+')

# =============================================================================
#
ion_energy_au = df_au.iloc[:,0]
el_power_au = df_au.iloc[:, 1]
nuc_power_au = df_au.iloc[:, 2]
# =============================================================================

# =============================================================================
#ion_energy_mg = df_mg.iloc[:,0]
#el_power_mg = df_mg.iloc[:, 1] * 0.1 # ev/ang
#nuc_power_mg = df_mg.iloc[:, 2] * 0.1 #eV/ang
#
#ion_energy_fe = df_fe.iloc[:,0]
#el_power_fe = df_fe.iloc[:, 1] * 0.1 # ev/ang
#nuc_power_fe = df_fe.iloc[:, 2] * 0.1 #eV/ang
# =============================================================================

ion_energy_o = df_o.iloc[:,0]
el_power_o = df_o.iloc[:, 1] # ev/ang
nuc_power_o = df_o.iloc[:, 2] #eV/ang

ion_energy_si = df_si.iloc[:,0]
el_power_si = df_si.iloc[:, 1] # ev/ang
nuc_power_si = df_si.iloc[:, 2] #eV/ang

In [ ]:

# energy threshold to apply electronic stopping power should be when dedx nuclear is > de/dx electronic
# =============================================================================
# =============================================================================
# for i in range(len(ion_energy_o)):
#     if el_power_o[i]/nuc_power_o[i] < 1.0:
#         continue
#     if el_power_o[i]/nuc_power_o[i] >+ 1.0:
#         print('the energy threshold is:' + str(ion_energy_o[i]))
# =============================================================================

In [ ]:
# points to interpolate to
# =============================================================================
xe_au = np.arange(0, len(el_power_au))
xi_au = np.arange(0, len(ion_energy_au))
# =============================================================================

# =============================================================================
# xe_mg = np.arange(0, len(el_power_mg))
# xi_mg = np.arange(0, len(ion_energy_mg))
#
# xe_fe = np.arange(0, len(el_power_fe))
# xi_fe = np.arange(0, len(ion_energy_fe))
# =============================================================================

xe_o = np.arange(0, len(el_power_o))
xi_o = np.arange(0, len(ion_energy_o))

xe_si = np.arange(0, len(el_power_si))
xi_si = np.arange(0, len(ion_energy_si))


In [ ]:
# interpolation
# =============================================================================
int_Se_au = scipy.interpolate.interp1d(xe_au, el_power_au)
# =============================================================================
# =============================================================================
# int_Se_mg = scipy.interpolate.interp1d(xe_mg, el_power_mg)
# int_Se_fe = scipy.interpolate.interp1d(xe_fe, el_power_fe)
# =============================================================================
int_Se_o = scipy.interpolate.interp1d(xe_o, el_power_o)
int_Se_si = scipy.interpolate.interp1d(xe_si, el_power_si)

# =============================================================================
int_IonE = scipy.interpolate.interp1d(xi_si, ion_energy_si)
# =============================================================================

# =============================================================================
elec_SRIM_fine_au = int_Se_au(np.arange(0, len(el_power_au)-1, 0.01))
# =============================================================================
# =============================================================================
# elec_SRIM_fine_mg = int_Se_mg(np.arange(0, len(el_power_mg)-1, 0.01))
# elec_SRIM_fine_fe = int_Se_fe(np.arange(0, len(el_power_fe)-1, 0.01))
# =============================================================================
elec_SRIM_fine_o = int_Se_o(np.arange(0, len(el_power_o)-1, 0.01))
elec_SRIM_fine_si = int_Se_si(np.arange(0, len(el_power_si)-1, 0.01))

# =============================================================================
ion_energy_SRIM_fine = int_IonE(np.arange(0, len(ion_energy_si)-1, 0.01))
# =============================================================================

In [ ]:
new_df = pd.DataFrame({'ion energy': ion_energy_SRIM_fine,
                       #'el power mg': elec_SRIM_fine_mg,
                       #'el power fe': elec_SRIM_fine_fe,
                       'el power o': elec_SRIM_fine_o,
                       'el power si': elec_SRIM_fine_si,
                       'el power au': elec_SRIM_fine_au})





#new_df.iloc[:,2] = new_df.iloc[:,2]*0.1
new_df.to_csv('Au_in_SiO2_elstoptable_500keV', sep =' ', index=None, header=None)

In [ ]:
plt.figure(0)
plt.plot(ion_energy_au, el_power_au, label = 'electronic stopping power', color = 'yellowgreen')
plt.plot(ion_energy_au, nuc_power_au, label = 'nuclear stopping power', color = 'blueviolet')
plt.legend()
plt.title("Stopping Power for Au per Ion Energy")
plt.xlabel('Ion Energy - eV')
plt.ylabel('Stopping Power - eV/Ang')

# plt.figure(1)
# plt.plot(ion_energy_mg, el_power_mg, label = 'electronic stopping power', color = 'yellowgreen')
# plt.plot(ion_energy_mg, nuc_power_mg, label = 'nuclear stopping power', color = 'blueviolet')
# plt.legend()
# plt.title("Stopping Power for Mg per Ion Energy")
# plt.xlabel('Ion Energy - eV')
# plt.ylabel('Stopping Power - eV/Ang')
#
# plt.figure(2)
# plt.plot(ion_energy_fe, el_power_fe, label = 'electronic stopping power', color = 'yellowgreen')
# plt.plot(ion_energy_fe, nuc_power_fe, label = 'nuclear stopping power', color = 'blueviolet')
# plt.legend()
# plt.title("Stopping Power for Fe per Ion Energy")
# plt.xlabel('Ion Energy - eV')
# plt.ylabel('Stopping Power - eV/Ang')

plt.figure(3)
plt.plot(ion_energy_o, el_power_o, label = 'electronic stopping power', color = 'yellowgreen')
plt.plot(ion_energy_o, nuc_power_o, label = 'nuclear stopping power', color = 'blueviolet')
plt.legend()
plt.title("Stopping Power for O in SiO2")
plt.xlabel('Ion Energy - eV')
plt.ylabel('Stopping Power - eV/Ang')


plt.figure(4)
plt.plot(ion_energy_si, el_power_si, label = 'electronic stopping power', color = 'yellowgreen')
plt.plot(ion_energy_si, nuc_power_si, label = 'nuclear stopping power', color = 'blueviolet')
plt.legend()
plt.title("Stopping Power for Si in SiO2")
plt.xlabel('Ion Energy - eV')
plt.ylabel('Stopping Power - eV/Ang')